In [3]:
import glob
import json
import datetime
from dateutil import parser

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML, Image, Video

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')

In [4]:
def convert_timeslice_to_buy_sell_curve(df1):
    df1 = df1.values.tolist()
    
    data = {
        "buy_curve": [],
        "sell_curve": [],
        "accepted_buy_volume": None,          # Accepted advance (e.g. block) orders
        "accepted_sell_volume": None,         # Accepted advance (e.g. block) orders
        #"nonaccepted_buy_volume": None,      # Only available before 2016
        #"nonaccepted_sell_volume": None,     # Only available before 2016
        "imported_volume_volume": None        # Can be negative
    }

    is_buy_or_sell_curve = False
    is_buy_curve = True

    last_recorded_volume = 0.0
    last_recorded_price = 0.0
    
    for i, row in enumerate(df1):
        if not is_buy_or_sell_curve:
            if row[0] == "Buy curve":
                is_buy_or_sell_curve = True
                
            elif row[0] == "Bid curve chart data (Volume for accepted blocks buy)":
                data["accepted_buy_volume"] = float(row[1])
            
            elif row[0] == "Bid curve chart data (Volume for accepted blocks sell)":
                data["accepted_sell_volume"] = float(row[1])
            
            #elif row[0] == "Bid curve chart data (Volume for non-accepted blocks buy)":
            #    data["nonaccepted_buy_volume"] = float(row[1])
            #
            #elif row[0] == "Bid curve chart data (Volume for non-accepted blocks sell)":
            #    data["nonaccepted_sell_volume"] = float(row[1])
            
            elif row[0] == "Bid curve chart data (Volume for net flows)":
                data["imported_volume_volume"] = float(row[1])
            
            else:
                continue
        else:
            if row[0] == "Sell curve":
                is_buy_curve = False
                continue

            elif row[0] == "Price value":
                price = float(row[1])
                # Wait for volume to record it

            elif row[0] == "Volume value":
                volume = float(row[1])
                
                # Record setting if significantly changed
                if (abs(volume - last_recorded_volume) > 1000    #MW
                    or abs(price - last_recorded_price) > 1): # EUR/MWh
                    
                    last_recorded_volume = volume
                    last_recorded_price = price

                    if is_buy_curve:
                        data["buy_curve"].append((price, volume))
                    else:
                        data["sell_curve"].append((price, volume))

            elif str(row[0]) == "nan":
                break 

            else:
                print(row)
                raise Exception("Unknown value in row %i: %s" % (i, row[0]))

    data["buy_curve"] = pd.DataFrame(data["buy_curve"], columns=["Price", "Volume"])
    data["sell_curve"] = pd.DataFrame(data["sell_curve"], columns=["Price", "Volume"])
    
    return data


In [5]:
def find_intersection(buy_curve, sell_curve):
    price = None
    volume = None

    i = 0; j = 0
    while i<len(buy_curve) and j<len(sell_curve):

        #print(i, j, buy_curve.iloc[i]["Volume"], sell_curve.iloc[j]["Volume"], buy_curve.iloc[i]["Price"], sell_curve.iloc[j]["Price"])

        if buy_curve.iloc[i]["Volume"] < sell_curve.iloc[j]["Volume"]:
            price = buy_curve.iloc[i]["Price"]
            volume = buy_curve.iloc[i]["Volume"]
            break
        else:
            if buy_curve.iloc[i]["Price"] < sell_curve.iloc[j]["Price"]:
                i += 1
            else:
                j += 1

    return volume, price

In [9]:
buy_and_sell_curves = []

files = glob.glob("mcp_data_report_*")
files = sorted(files, key=lambda file: parser.parse(file[16:26], dayfirst=True)) # Uten data/
#files = sorted(files, key=lambda file: parser.parse(file[21:31], dayfirst=True))
#files = files[-2:]

for index, file in enumerate(files):
    progress = "{}/{} ".format(index+1, len(files))
    
    try:
        df = pd.read_excel(file)
        
        for i in range(24):
            date_time_str = df.iloc[:,(i*2):(i*2+2)].columns[1]
            date_time = parser.parse(date_time_str[:19], dayfirst=True)
            data = convert_timeslice_to_buy_sell_curve(df.iloc[:,(i*2):(i*2+2)])
            
            data["buy_curve"]["Volume"] = data["buy_curve"]["Volume"] + data["accepted_buy_volume"] - data["imported_volume_volume"]
            data["sell_curve"]["Volume"] = data["sell_curve"]["Volume"] + data["accepted_sell_volume"]
            
            volume, price = find_intersection(data["buy_curve"], data["sell_curve"])
            
            # New data
            data["volume"] = volume
            data["price"] = price
            data["date_time"] = date_time
            
            buy_and_sell_curves.append(data)
            
    except Exception as e:
        print(e)
        print(progress + file + " FAILED!")
    else:
        print(progress + file)
        #print(buy_and_sell_curves[index*24][0], buy_and_sell_curves[index*24][3])
        #print(buy_and_sell_curves[index*24+1][0], buy_and_sell_curves[index*24+1][3])
    
buy_and_sell_curves = sorted(buy_and_sell_curves, key=lambda item: item["date_time"])
#buy_and_sell_curves[17]

1/139 mcp_data_report_27-08-2019-00_00_00.xls
2/139 mcp_data_report_28-08-2019-00_00_00.xls
3/139 mcp_data_report_29-08-2019-00_00_00.xls
4/139 mcp_data_report_30-08-2019-00_00_00.xls
5/139 mcp_data_report_31-08-2019-00_00_00.xls
6/139 mcp_data_report_01-09-2019-00_00_00.xls
7/139 mcp_data_report_02-09-2019-00_00_00.xls
8/139 mcp_data_report_03-09-2019-00_00_00.xls
9/139 mcp_data_report_04-09-2019-00_00_00.xls
10/139 mcp_data_report_05-09-2019-00_00_00.xls
11/139 mcp_data_report_06-09-2019-00_00_00.xls
12/139 mcp_data_report_07-09-2019-00_00_00.xls
13/139 mcp_data_report_08-09-2019-00_00_00.xls
14/139 mcp_data_report_09-09-2019-00_00_00.xls
15/139 mcp_data_report_10-09-2019-00_00_00.xls
16/139 mcp_data_report_11-09-2019-00_00_00.xls
17/139 mcp_data_report_12-09-2019-00_00_00.xls
18/139 mcp_data_report_13-09-2019-00_00_00.xls
19/139 mcp_data_report_14-09-2019-00_00_00.xls
20/139 mcp_data_report_15-09-2019-00_00_00.xls
21/139 mcp_data_report_16-09-2019-00_00_00.xls
22/139 mcp_data_report

In [10]:
new_buy_and_sell_curves = buy_and_sell_curves

In [11]:
def myconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
    if isinstance(o, pd.DataFrame):
        return o.values.tolist()

j = json.dumps(new_buy_and_sell_curves, default = myconverter)

with open('gitignore/data-justnew.json', 'w') as file:
    file.write(j)

In [15]:
!cp -f gitignore/data.json gitignore/data-old.json

In [13]:
with open('gitignore/data-old.json', 'r') as file:
    old_buy_and_sell_curves = json.load(file)
    for old_buy_and_sell_curve in old_buy_and_sell_curves:
        old_buy_and_sell_curve["date_time"] = parser.parse(old_buy_and_sell_curve["date_time"], yearfirst=True)
        old_buy_and_sell_curve["buy_curve"] = pd.DataFrame(old_buy_and_sell_curve["buy_curve"], columns=["Price", "Volume"])
        old_buy_and_sell_curve["sell_curve"] = pd.DataFrame(old_buy_and_sell_curve["sell_curve"], columns=["Price", "Volume"])
#old_buy_and_sell_curves

In [14]:
def myconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
    if isinstance(o, pd.DataFrame):
        return o.values.tolist()

buy_and_sell_curves_org = old_buy_and_sell_curves
buy_and_sell_curves_org += new_buy_and_sell_curves
j = json.dumps(buy_and_sell_curves_org, default = myconverter)

with open('gitignore/data.json', 'w') as file:
    file.write(j)